In [ ]:
K8S_PROXY_ADDR='127.0.0.1:8001'
K8S_NAMESPACE='mdt'
APP_NAME='pyspark-classification'
MODEL_NAME='clipper-pyspark-predict'
REPO_URL='658391232643.dkr.ecr.us-west-2.amazonaws.com'
VERSION = 1

SPARK_URL='ae092775dc67111e8aee806444a30499-1743429263.us-west-2.elb.amazonaws.com'
SPARK_APP='Clipper'
HADOOP_URL='ad95fe885c37011e8aee806444a30499-1181034928.us-west-2.elb.amazonaws.com'


Connect to clipper and register App

In [ ]:
from clipper_admin import ClipperConnection, KubernetesContainerManager
from clipper_admin.deployers import python as python_deployer
manager = KubernetesContainerManager(
    kubernetes_proxy_addr=K8S_PROXY_ADDR, namespace=K8S_NAMESPACE)
clipper_conn = ClipperConnection(manager)
clipper_conn.connect()

clipper_conn.register_application(
   name = APP_NAME, input_type = 'doubles', default_output = '0', slo_micros = 100000000)

Create a prediction function and deploy it to clipper

In [ ]:
import pyspark
from clipper_admin.deployers.pyspark import deploy_pyspark_model
from pyspark.ml import Pipeline
from pyspark.ml import PipelineModel
from pyspark.sql import SparkSession
import numpy as np

sparkUrl = "spark://{}:7077".format(SPARK_URL)
conf = pyspark.SparkConf().setAppName(SPARK_APP).setMaster(sparkUrl)
sc = pyspark.SparkContext(conf=conf)

spark = SparkSession.builder.master(SPARK_URL) \
    .appName(SPARK_APP) \
    .getOrCreate()

# Loading a training dataset omitted...
#pipeline = Pipeline.read().load("hdfs://{}:9000/tmp/classification-pipeline".format(SPARK_URL))
model = PipelineModel.read().load("hdfs://{}:9000/tmp/classification-model".format(SPARK_URL))

# Note that this function accesses the trained PySpark model via an explicit
# argument, but other state can be captured via closure capture if necessary.
def predict(spark, model, inputs):
    return [str(model.transform(x)) for x in inputs]

deploy_pyspark_model(
    clipper_conn,
    name=MODEL_NAME,
    input_type="doubles",
    func=predict,
    pyspark_model=model,
    sc=sc)

Link the model to application

In [ ]:
clipper_conn.link_model_to_app(app_name=APP_NAME, model_name=MODEL_NAME)

In [ ]:
import requests, json, numpy as np
def predict():
    headers = {"Content-type": "application/json"}
    data=json.dumps({"input": [5.9, 3.0, 5.1, 1.8]})
    print(data)
    url = "http://{}/api/v1/namespaces/{}/services/query-frontend-at-default-cluster:1337/proxy/{}/predict".format(K8S_PROXY_ADDR,K8S_NAMESPACE,APP_NAME)
    res = requests.post(url, headers=headers, data=data)
    return res.json()

print(predict())